# Visualise Benchmarking Results

In [29]:
import numpy as np
import pandas as pd
from lets_plot import *

LetsPlot.setup_html(no_js=True)

In [30]:
res_py = pd.read_csv("./results_py.csv")
res_py["model"] = np.where(res_py["method"] == "feols", "Gaussian", "Poisson")
res_py["method"] = "pyfixest"
res_other = pd.read_csv("./results_all.txt")
res_all = pd.concat([res_py, res_other], axis=0)

In [31]:
res_all.method.unique()

array(['pyfixest', 'fepois', 'glmmboot', 'feglm (alpaca)', 'ppmlhdfe',
       'fenegbin', 'glmnb', 'nbreg', 'feglm (fixest)', 'logit', 'feols',
       'lfe', 'reghdfe', 'FixedEffectModels'], dtype=object)

In [32]:
res_agg = (
    res_all.groupby(["method", "n_obs", "G", "model"]).mean()["time"].reset_index()
)
res_agg["G"] = res_agg["G"].apply(lambda x: f"{x} FE")
res_agg["method"] = pd.Categorical(
    res_agg["method"],
    [
        "pyfixest",
        "feols",
        "reghdfe",
        "lfe",
        "FixedEffectModels",
        "fepois",
        "glmmboot",
        "ppmlhdfe",
        "feglm (alpaca)",
    ],
)

In [33]:
plot_ols = (
    ggplot(
        res_agg[res_agg["model"] == "Gaussian"],
        aes(x="n_obs", y="time", color="method"),
    )
    + geom_line()
    + geom_point()
    + facet_wrap("G", nrow=1)
    + scale_x_discrete()
    + scale_y_continuous(trans="log10", limits=(0, 120))
    + ylab("Time in Seconds")
    + xlab("Number of Observations")
    + ggtitle("Fixest Standard Benchmark for OLS")
    + ggsize(1000, 500)
)
ggsave(plot_ols, filename="benchmarks_ols.svg")


plot_ols

In [35]:
plot_poisson = (
    ggplot(
        res_agg[(res_agg["model"] == "Poisson") & (res_agg.n_obs < 1e07)],
        aes(x="n_obs", y="time", color="method"),
    )
    + geom_line()
    + geom_point()
    + facet_wrap("G", nrow=1)
    + scale_x_discrete()
    + scale_y_continuous(trans="log10", limits=(0, 120))
    + ylab("Time in Seconds")
    + xlab("Number of Observations")
    + ggtitle("Fixest Standard Benchmark for Poisson Regression")
    + ggsize(1000, 500)
)

ggsave(plot_poisson, filename="benchmarks_poisson.svg")

plot_poisson